# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [1]:
import ee
import geemap
ee.Initialize(project='ee-jsumara')
aoi = ee.Geometry.Rectangle([19.9762,50.092,19.9889,50.0986]) 
Map = geemap.Map(center=(50.095, 19.985), zoom=15)
Map.addLayer(aoi, {'color': 'red'}, "AOI")
Map

Map(center=[50.095, 19.985], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [ ]:
landsat_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2013-04-01', '2013-08-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .sort('CLOUD_COVER') \
    .map(lambda img: img.clip(aoi))


landsat_2023 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2023-04-01', '2023-08-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .sort('CLOUD_COVER') \
    .map(lambda img: img.clip(aoi))


image_2013 = landsat_2013.first()
tb_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149).rename('TB')

image_2023 = landsat_2023.first()
tb_2023 = image_2023.select('ST_B10').multiply(0.00341802).add(149).rename('TB') 

tb_vis_params = {
    'min': 290,  # min temp
    'max': 320,  # max temp
    'palette': ['blue', 'green', 'yellow', 'red'] 
}

Map = geemap.Map()
Map.centerObject(aoi, zoom=15)
Map.addLayer(tb_2013, tb_vis_params, 'TB 2013')
Map.addLayer(tb_2023, tb_vis_params, 'TB 2023')
Map

Map(center=[50.095300097386605, 19.982549999987466], controls=(WidgetControl(options=['position', 'transparent…

## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [5]:
corine_2018 = ee.Image("COPERNICUS/CORINE/V20/100m/2018").clip(aoi)
corine_2012 = ee.Image("COPERNICUS/CORINE/V20/100m/2012").clip(aoi)

median_2012 = corine_2012.select('landcover').reduceRegion(
    reducer=ee.Reducer.median(),
    geometry=aoi,
    scale=100,
    maxPixels=1e9
)

median_2018 = corine_2018.select('landcover').reduceRegion(
    reducer=ee.Reducer.median(),
    geometry=aoi,
    scale=100,
    maxPixels=1e9
)

median_2012_value = median_2012.getInfo()['landcover']
median_2018_value = median_2018.getInfo()['landcover']
print(median_2012_value, median_2018_value)

112 112


## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [ ]:
# Create emissivity image

emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

emissivity = corine_2018.select('landcover').remap(list(emissivity_dict.keys()), list(emissivity_dict.values())).rename('emissivity')

emissivity

In [8]:
image_2013 = tb_2013.addBands(emissivity).addBands(tb_2013)
image_2023 = tb_2023.addBands(emissivity).addBands(tb_2023)
collection = ee.ImageCollection([image_2013, image_2023])
collection

## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [12]:
# Calculate LST
def compute_lst(image):
    lst = image.expression(
        ' TB / (1 + (lambda_ * TB / c2) * log(eps))',
        {
            'TB': image.select('TB'),
            'eps': image.select('emissivity'),
            'lambda_': 0.014387,
            'c2': 14388.0
        }
    ).rename('LST')
    
    return image.addBands(lst)

collection_lst = collection.map(compute_lst)
collection_lst

## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [13]:
# Visualize LST
lst_vis_params = {
    'min': 290,  # Minimalna wartość temperatury w Kelvinach
    'max': 320,  # Maksymalna wartość temperatury w Kelvinach
    'palette': ['blue', 'green', 'yellow', 'red']  # Skala kolorów
}
Map2 = geemap.Map()
Map2.centerObject(aoi, zoom=15)
Map2.addLayer(collection_lst.select('LST').first().select('LST'), lst_vis_params, 'LST 2013')
Map2.addLayer(collection_lst.sort('system:time_start', False).first().select('LST'), lst_vis_params, 'LST 2023')
Map2

Map(center=[50.09530009742444, 19.982549999985554], controls=(WidgetControl(options=['position', 'transparent_…

## Step 7: (Optional) Analyze statistics by land cover class

## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade